In [10]:
import pandas as pd
import numpy as np

df = pd.DataFrame(
    {
        "data": np.random.rand(100).astype(np.float32),
    }
)
df

,data
0,0.340275
1,0.875363
2,0.193638
3,0.496009
4,0.442885
...,...
95,0.250776
96,0.395493
97,0.278861
98,0.828517


In [11]:
import pyarrow as pa
import pyarrow.dataset as pda

chunk_size = 10
output_path = "output"

table = pa.Table.from_pandas(df)

pda.write_dataset(
    table,
    output_path,
    format="parquet",
    max_rows_per_file=chunk_size,
    max_rows_per_group=chunk_size,
    # existing_data_behavior="overwrite_or_ignore",
    existing_data_behavior="error",
    partitioning_flavor="hive",
)

In [16]:
import pyarrow.parquet as pq

writer = pq.ParquetWriter(
    "part-0.parquet",
    table.schema,
    compression="snappy",
)
writer.write_table(table, row_group_size=chunk_size)
writer.close()

In [18]:
pq.read_table('part-0.parquet').to_pandas()

,data
0,0.340275
1,0.875363
2,0.193638
3,0.496009
4,0.442885
...,...
95,0.250776
96,0.395493
97,0.278861
98,0.828517
